In [ ]:
#確認：B列だけか？他の列はあり得るか？　#J業、B、E、J、M
#確認：色並び替えは縦一列でいいか、#一列でOK
#確認　物理地一覧は一部だけ？シートをまたぐ可能性もある、変更1,2,3で数字が降られる

#ソフト開発
#定数いじったものが出てくる
#型変更、同じ定数だけど型違う

#課題
#・コードBOX共有、jupyter置き場共有
# 藤井さん確認

#1/30に確認

#笹岡
#プログラム
#抜け漏れがないかの確認
#全シートに適用する
#定数一覧B列以外の列にも対応する→開始と終了は○○一覧をもとに決定できる
#シートをマタギに対応

In [3]:
import openpyxl
from copy import copy
from openpyxl.styles import PatternFill
from openpyxl.styles import Font, PatternFill, Border, Alignment


#定数一覧の範囲
paras = 7 #B列の7行
parae = 151 #B列の151行

# 元のファイルを読み込む
file_path = 'LB71_IRS23-00445_A052_LB68_IRS23-00669_B027_1_CRC-7BEBDE36_Classified.xlsx'
workbook = openpyxl.load_workbook(file_path, data_only=True)


# 処理1: 「変更1」シートのB列のデータを背景色に基づいて並び替える
sheet = workbook['変更1']
data_with_colors = []
for row in range(paras, parae):  # B列の7行目から151行目まで
    cell = sheet.cell(row, 2)  # B列
    data_with_colors.append((cell.value, cell.fill.start_color.rgb if cell.fill.fill_type else None))

data_with_colors.sort(key=lambda x: x[1] or '')  # 背景色で並び替え

# 新しいワークブックを作成し、「並び替え」シートにデータを出力
new_workbook = openpyxl.Workbook()
sort_sheet = new_workbook.create_sheet('定数並び替え一覧', 0)
for index, (value, color) in enumerate(data_with_colors, start=7):
    cell = sort_sheet.cell(row=index, column=2)  # B列
    cell.value = value
    if color:
        cell.fill = PatternFill(start_color=color, fill_type="solid")
        
# 処理2: 「変更1」シートのA列からR列のデータと色情報を「生データ」シートにコピー
temp_sheet = new_workbook.create_sheet('生データ', 1)
for row in range(1, sheet.max_row + 1):
    for col in range(1, 19):  # A列(1)からR列(18)まで
        original_cell = sheet.cell(row, col)
        new_cell = temp_sheet.cell(row, col)
        new_cell.value = original_cell.value
        if original_cell.has_style:  # セルにスタイルがあればコピー
            new_cell.font = copy(original_cell.font)
            new_cell.border = copy(original_cell.border)
            new_cell.fill = copy(original_cell.fill)
            new_cell.number_format = copy(original_cell.number_format)
            new_cell.protection = copy(original_cell.protection)
            new_cell.alignment = copy(original_cell.alignment)

# 処理3: 「仮」シートのデータを条件に基づいて「生データ_抽出」シートにコピー
max_row_temp_sheet = temp_sheet.max_row
temp2_sheet = new_workbook.create_sheet('生データ_抽出', 2)
for row in range(1, max_row_temp_sheet + 1):
    if row + 2 <= max_row_temp_sheet and temp_sheet.cell(row + 2, 1).value == '変更理由':
        temp2_sheet.cell(row, 1).value = temp_sheet.cell(row, 1).value
        
# 処理4: 「生データ_抽出」シートのA列データの隣の列に行数を追記
max_row_temp2_sheet = temp2_sheet.max_row
for row in range(1, max_row_temp2_sheet + 1):
    current_cell = temp2_sheet.cell(row, 1)
    if current_cell.value is not None:
        count = 0
        for next_row in range(row + 1, max_row_temp2_sheet + 1):
            if temp2_sheet.cell(next_row, 1).value is not None:
                break
            count += 1
        temp2_sheet.cell(row, 2).value = count

# 処理5: 「定数並び替え一覧」と「生データ_抽出」シートを比較し、「物理値並び替え一覧」シートに一致するデータを追記
temp3_sheet = new_workbook.create_sheet('物理値並び替え一覧')
sort_data = [cell.value for cell in sort_sheet['B'][paras-1:parae-1]]  # B7 to B151 in '定数並び替え一覧' sheet
temp2_data = [cell.value for cell in temp2_sheet['A'][:max_row_temp2_sheet]]  # Up to A3883 in '生データ_抽出' sheet

output_row_index = 156  # B列の出力行の開始位置
for data in sort_data:
    if data in temp2_data:
        # 一致するデータを「物理値並び替え一覧」シートに出力
        temp3_sheet.cell(row=output_row_index, column=1).value = data

        # 「生データ_抽出」シートのB列から一致するデータの行数を取得
        match_row_index = temp2_data.index(data)
        space_count = temp2_sheet.cell(row=match_row_index + 1, column=2).value or 0
        
        temp3_sheet.cell(row=output_row_index, column=2).value = space_count

        # 次の一致するデータのために行インデックスを更新
        output_row_index += space_count + 1  # 空白行の数と現在のデータ行を加算

# 最終的なワークブックを保存
final_workbook_path = '中間成果物.xlsx'
new_workbook.save(final_workbook_path)
final_workbook_path

'中間成果物.xlsx'

In [6]:
# セルのプロパティをコピーする関数を再定義し、正しいインポートを追加する
def copy_cell_properties(source_cell, target_cell):
    # 値をコピーする
    target_cell.value = source_cell.value

    # フォントのプロパティをコピーする
    if source_cell.font:
        target_cell.font = Font(name=source_cell.font.name, 
                                size=source_cell.font.size,
                                bold=source_cell.font.bold, 
                                italic=source_cell.font.italic,
                                vertAlign=source_cell.font.vertAlign, 
                                underline=source_cell.font.underline,
                                strike=source_cell.font.strike, 
                                color=source_cell.font.color)

    # 塗りつぶしのプロパティをコピーする
    if source_cell.fill:
        target_cell.fill = PatternFill(fill_type=source_cell.fill.fill_type,
                                       start_color=source_cell.fill.start_color,
                                       end_color=source_cell.fill.end_color)

    # 枠線のプロパティをコピーする
    if source_cell.border:
        target_cell.border = Border(left=source_cell.border.left, 
                                    right=source_cell.border.right,
                                    top=source_cell.border.top, 
                                    bottom=source_cell.border.bottom,
                                    diagonal=source_cell.border.diagonal,
                                    diagonal_direction=source_cell.border.diagonal_direction,
                                    outline=source_cell.border.outline,
                                    vertical=source_cell.border.vertical,
                                    horizontal=source_cell.border.horizontal)

    # 配置のプロパティをコピーする
    if source_cell.alignment:
        target_cell.alignment = Alignment(horizontal=source_cell.alignment.horizontal,
                                          vertical=source_cell.alignment.vertical,
                                          text_rotation=source_cell.alignment.text_rotation,
                                          wrap_text=source_cell.alignment.wrap_text,
                                          shrink_to_fit=source_cell.alignment.shrink_to_fit,
                                          indent=source_cell.alignment.indent)


# 元のファイルを読み込む
workbook = openpyxl.load_workbook(final_workbook_path)
        
# 新しいワークブックとシートを初期化する
new_workbook = openpyxl.Workbook()
new_sheet = new_workbook.active
new_sheet.title = '物理値並び替え一覧'

# '定数並び替え一覧'シートにアクセスする
sheet_constants = workbook['定数並び替え一覧']

# 再度2つのシートにアクセスする
sheet_raw_data = workbook['生データ']
sheet_sorted_values = workbook['物理値並び替え一覧']

# 元のシートから新しいシートに各セルをコピーする
for row in sheet_constants.iter_rows():
    for cell in row:
        new_cell = new_sheet.cell(row=cell.row, column=cell.column)
        copy_cell_properties(cell, new_cell)
        
# '物理値並び替え一覧'シートを繰り返し処理し、'生データ'と比較する
for row, cell in enumerate(sheet_sorted_values['A'], start=1):
    if cell.value:  # セルに文字列があるかどうかをチェックする
        for raw_row, raw_cell in enumerate(sheet_raw_data['A'], start=1):
            if cell.value == raw_cell.value:
                # 一致する文字列を出力し、A列のセルのプロパティをコピーする
                copy_cell_properties(cell, new_sheet.cell(row=row, column=1))
                # '物理値並び替え一覧'のB列の値とセルのプロパティを出力する
                copy_cell_properties(sheet_sorted_values.cell(row=row, column=2), new_sheet.cell(row=row, column=2))
                
                # '物理値並び替え一覧'のB列の値を新しいファイルのB列に出力する
                #new_sheet.cell(row=row, column=2, value=sheet_sorted_values.cell(row=row, column=2).value)

                # '生データ'の次の行のA列～Q列とセルのプロパティを出力する
                if raw_row < sheet_raw_data.max_row:
                    for gyo in range(sheet_sorted_values.cell(row=row, column=2).value+1):
                        #gyo = gyo + 1
                        for retsu in range(18):
                            retsu = retsu + 1
                            raw_next_cell = sheet_raw_data.cell(row=raw_row + gyo, column=retsu)
                            new_next_cell = new_sheet.cell(row=row + gyo, column=retsu)
                            copy_cell_properties(raw_next_cell, new_next_cell)
                break


# 最終的なワークブックを保存
updated_output_file_path_advanced_b = '変更後.xlsx'
new_workbook.save(updated_output_file_path_advanced_b)

updated_output_file_path_advanced_b                


'変更後.xlsx'